In [0]:
import os
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import _pickle as cPickle

from datetime import datetime

from sklearn.svm import SVR   
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


In [0]:
!unzip jsonLatest.zip -d jsonData/
dirpath=os.path.join(os.getcwd(),'jsonData/jsonLatest')

In [0]:
#Fetch Data and Make a Dataframe
####################################################

json_list=[]
stock_set=set()

for root, dirs, files in os.walk(dirpath):
  files.sort()
  for file in files:
      if file.endswith(".json"):

          filepath=os.path.join(dirpath,file)

          with open(os.path.join(filepath)) as f:

              data = f.read()

              day_json=json.loads(data.strip())

              for key in day_json.keys():
                  if key!='id':
                      x=day_json[key]
                      x['date']=day_json['id']
                      # x['taiex']=day_json['taiex']['close']
                      x['id']=key
                      if day_json[key]['open']!="NULL" and day_json[key]['close']!="NULL" and day_json[key]['low']!="NULL" and day_json[key]['high']!="NULL" and day_json[key]['adj_close']!="NULL":
                        json_list.append(x)
                        stock_set.add(key)


df=pd.DataFrame.from_records(json_list)

In [0]:

#changing the datatype of columns
df[['adj_close', 'close','high','low','open']] = df[['adj_close', 'close','high','low','open']].astype(float)
df['date'] =  pd.to_datetime(df['date'],format='%Y-%m-%d')
df['volume'] = df['volume'].apply(pd.to_numeric)
# df.describe()

In [0]:
#Plot open low high close
# dff=df[df['id']=='067383']
# # dff.sort_values(by=['date'])
# # dff.iloc[0:5,:]
# # dff.describe()
# plt.plot(dff['date'], dff['open'], color='b')         # short color code (rgbcmyk)
# plt.plot(dff['date'], dff['high'], color='k')        # Grayscale between 0 and 1
# plt.plot(dff['date'], dff['close'], color='g')        # specify color by name
# plt.plot(dff['date'], dff['low'], color='r')  
# # plt.plot(dff['date'], dff['adj_close'], color='#FFDD44')     

# plt.show()

In [0]:

stock_list=list(stock_set)
output_list=[]

for stock_id in stock_list[:100]:
  try:
    dff=df[df['id']==stock_id]
    
    #Preparing the data for Single Stock
    #####################################################
    x={}
    
    #finding new features
    dff["20d_ac"] = np.round(dff["adj_close"].rolling(window = 20, center = False).mean(), 2)
    dff["10d_ac"] = np.round(dff["adj_close"].rolling(window = 10, center = False).mean(), 2)
    dff["5d_ac"] = np.round(dff["adj_close"].rolling(window = 5, center = False).mean(), 2)
    
    print("id:"+str(stock_id))
    
    dff["20d_l"] = np.round(dff["low"].rolling(window = 20, center = False).mean(), 2)
    dff["10d_l"] = np.round(dff["low"].rolling(window = 10, center = False).mean(), 2)
    dff["5d_l"] = np.round(dff["low"].rolling(window = 5, center = False).mean(), 2)
    
    dff["20d_h"] = np.round(dff["high"].rolling(window = 20, center = False).mean(), 2)
    dff["10d_h"] = np.round(dff["high"].rolling(window = 10, center = False).mean(), 2)
    dff["5d_h"] = np.round(dff["high"].rolling(window = 5, center = False).mean(), 2)
    
    dff["20d_o"] = np.round(dff["open"].rolling(window = 20, center = False).mean(), 2)
    dff["10d_o"] = np.round(dff["open"].rolling(window = 10, center = False).mean(), 2)
    dff["5d_o"] = np.round(dff["open"].rolling(window = 5, center = False).mean(), 2)
    
    dff["mid_ol"]=(dff["open"]+dff["low"])/2
    dff["mid_hc"]=(dff["high"]+dff["close"])/2
    dff["mid_cadjc"]=(dff["adj_close"]+dff["close"])/2
    
    
    #target
    dff['ret_2'] = dff['adj_close'].pct_change(2)

    x["id"]=stock_id
    
    #SVR for predicting the next low , mid_hc , mid_ol
    ######################################################
    #Preparing Data

    features = ['20d_l','5d_l','open','mid_ol','mid_cadjc','ret_2']
    
    X = dff[features]    
    y = dff['low']
    
    total_rows=len(dff.index)
    n=total_rows-1
    X_train = X.iloc[21:n-1,:]
    y_train = y.iloc[21:n-1]              

    X_test = X.iloc[n,:]
    y_test = y.iloc[n]

    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

    svr_rbf.fit(X_train,y_train)
    print('success')
    y_rbf = svr_rbf.predict(np.asarray(X_test).reshape(1,-1))
    
    predicted_low=y_rbf[0]
    actual_low=y_test
    

    #   #mid_ol (open price)
    features = ['10d_l','20d_l','5d_l','open','mid_cadjc','20d_o','10d_o','5d_o','low','ret_2']
    X = dff[features]    
    y = dff['mid_ol']
    
    total_rows=len(dff.index)
    n=total_rows-1
    X_train = X.iloc[21:n-1,:]
    y_train = y.iloc[21:n-1]
    X_test = X.iloc[n,:]
    y_test = y.iloc[n]
    
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_rbf.fit(X_train,y_train)
    
    #   print('success')
    y_rbf = svr_rbf.predict(np.asarray(X_test).reshape(1,-1))
    
    predicted_ol=y_rbf[0]
    actual_ol=y_test
    
    features = ['20d_h','10d_h','5d_h','mid_cadjc','10d_ac','20d_ac','5d_ac','close','high','ret_2']
    X = dff[features]    
    y = dff['mid_hc']

    total_rows=len(dff.index)
    n=total_rows-1

    X_train = X.iloc[21:n,:]
    y_train = y.iloc[21:n]
    X_test = X.iloc[n,:]
    y_test = y.iloc[n]

    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

    svr_rbf.fit(X_train,y_train)
    y_rbf = svr_rbf.predict(np.asarray(X_test).reshape(1,-1))
    
    predicted_hc=y_rbf[0]
    actual_hc=y_test
                
    #Last Day's Data
    last_day=dff['adj_close']
    last_day_adj=dff.iloc[n-1]
    
    

    x['open_price']=last_day_adj[0]-(predicted_ol-predicted_low) #2
    x['close_high_price']=x['open_price']+(predicted_hc-predicted_ol)  #7

    #swap 
    if(x['close_high_price']<x['open_price']):
      t=x['close_high_price']
      x['close_high_price']=x['open_price']
      x['open_price']=t
    
    
    #     #RandomForestClassifier : For predicting Buy or Sell
    #     ####################################################
    #     #Preparing Data
    features = ['adj_close','volume','10d_o','20d_o','5d_o','mid_cadjc','mid_ol','mid_hc']

    le = preprocessing.LabelEncoder()
    
    dff['UpDown'] = dff['ret_2']
    dff.loc[dff['UpDown'] >= 0,'UpDown']=1 
    dff.loc[dff['UpDown'] < 0,'UpDown']=0 
    dff['UpDown'] = dff['UpDown'].apply(pd.to_numeric)
    
    X = dff[features]    
    y = dff['UpDown']

    total_rows=len(dff.index)
    n=total_rows-1

    X_train = X.iloc[21:n-1,:]
    y_train = y.iloc[21:n-1]
    X_test = X.iloc[n,:]
    y_test = y.iloc[n]

    #Model Random Forest
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(X_train, y_train)

    predicted_score = clf.predict(np.asarray(X_test).reshape(1,-1))
    

    if(predicted_score>0.5):
      x['type']='buy'
    else:
      x['type']='short'
    x['weight']=1
    x['life']=3
      
          
    output_list.append(x)

  except Exception as e:
    print("Id:"+str(stock_id)+str(e))

  
sortedlist = sorted(output_list, key=lambda x: x['close_high_price']-x['open_price'], reverse=True)  
    



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

id:078910


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

success


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

success


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

id:06434P


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
# f=open("write_file", "w+")
# f.write(str(sortedlist))
# f.close()

trade_stock=sortedlist[:3]
for stock in trade_stock:
  stock['weight']=stock['weight']/3

print(str(trade_stock))

f=open("decision_file","w")
f.write(str(trade_stock))
f.close()


In [39]:
# weight_list=[]
# f=open("decision_file","r")
# x=f.read()
# print(type(x))

<class 'str'>


In [0]:
#Test Runs
#Preparing the data for Single Stock
#####################################################

print("0050")
dff=df[df['id']=='0050']

# #changing the datatype of columns
# dff[['adj_close', 'close','high','low','open']] = dff[['adj_close', 'close','high','low','open']].astype(float,copy=False)
# dff['volume'] = dff['volume'].apply(pd.to_numeric)
# dff['date'] =  pd.to_datetime(dff['date'],format='%Y-%m-%d')

#finding new features
dff["60d"] = np.round(dff["adj_close"].rolling(window = 60, center = False).mean(), 2)
dff["20d"] = np.round(dff["adj_close"].rolling(window = 20, center = False).mean(), 2)

#target
dff['ret_2'] = dff['adj_close'].pct_change(2)

le = preprocessing.LabelEncoder()
dff['UpDown'] = dff['ret_2']
dff.loc[dff['UpDown'] >= 0,'UpDown']=1 
dff.loc[dff['UpDown'] < 0,'UpDown']=0 
# dff.UpDown = le.fit(dff.UpDown).transform(dff.UpDown)
dff['UpDown'] = dff['UpDown'].apply(pd.to_numeric)


In [17]:
#SVR for predicting the adj_close
######################################################
#Preparing Data

features = ['close','60d','20d','low','high','open','ret_2']
X = dff[features]    
y = dff['adj_close']

X_train = X.iloc[61:200,:]
y_train = y.iloc[61:200]              
# print(X_train.shape)
# X_test = X.iloc[200:,:]    
# y_test = y.iloc[200:]
X_test = X.iloc[201,:]
y_test = y.iloc[201]
# print("Xtest "+str(X_test.shape))
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

svr_rbf.fit(X_train,y_train) 
y_rbf = svr_rbf.predict(np.asarray(X_test).reshape(1,-1))

print("Prediction")
print(y_rbf[0])
print("Original")
# print(str(y_test.iloc[201:205].tolist()))
print(y_test)


Prediction
85.70230449881865
Original
85.5


In [9]:
#RandomForestClassifier : For predicting Buy or Sell
####################################################
#Preparing Data
features = ['adj_close','volume','60d','20d']

X = dff[features]    
y = dff['UpDown']

X_train = X.iloc[61:200,:]
y_train = y.iloc[61:200]              

X_test = X.iloc[200:,:]    
y_test = y.iloc[200:]

# print(dff.tail())

#Model Random Forest
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf.fit(X_train, y_train)


predicted_scores = clf.predict(X_test)



accuracy = clf.score(X_test, y_test)
print(accuracy)

0.8807947019867549
